# Basic Optimization of Search Space with PySHAC

PySHAC is a lightweight library that can be used in any case where there exists a large search space, either discrete or continious, and we need to evaluate parameters of the search space to optimize some objective function.

An easy use case is hyper parameter optimization, and a more advanced case is to perform Neural Architecture Search. In either case, we deal with large search spaces - in terms of hyper parameter combinations and in terms of the model architecture choices that we can construct.

Let's first try pyshac out on a small search space - finding two parameters (which can take continious values) and try to find a linear combination to maximize some objective.

In [1]:
import numpy as np
import pyshac

## Task
Lets try to find two parameters - `x` and `y` such that the linear combination such that $$f(x, y) = 2 * x - y$$

Lets consider that `f(x, y)` must be equal to some number (or at least close to it). This is an un-constrained problem, with a large number of possible answers. 

Let's assume that f(x, y) must be close to some number, say 4.0. This is arbitrary, and can be changed below. Now we need some notion of a loss metric, to detail how far `f` is from the value of 4.0. Let's use the most common metric of squared error.

In [2]:
# The value that `f` needs to get close to
value = 4.0

# Defining Hyper Parameters
First, lets define the search space for the function. As per our function, we need two parameters `x` and `y` so that the function can compute its value.

A Hyper Parameter is the basis of defining a search space in PySHAC. There are 3 specific parameters that we can declare :

1) `DiscreteHyperParameter`: These are used to declare search spaces that take discrete values. These are most useful when defining hyper parameter optimization or architecture search spaces.

2) `UniformContiniousHyperParameter` / `NormalContiniousHyperParameter`: These are continious hyper parameters that can sample from some distribution. Usually, we would use the Uniform version.



In [3]:
# define the parameters
param_x = pyshac.UniformContinuousHyperParameter('x', -5.0, 5.0)
param_y = pyshac.UniformContinuousHyperParameter('y', -2.0, 2.0)

parameters = [param_x, param_y]

# Evaluation functions
PySHAC will require an objective function that it tries to optimize (can be either maximization or minimization).

PySHAC's evaluation functions are quite simple to setup. An evaluation function has two inputs that it has to accept : 

1) id: An integer value representing the id of the thread or the executor. This is useful to decide which GPU to place operations on and compute values when using Tensorflow or Keras.

2) parameters: An ordered dictionary of HyperParameters, in the order that they will be supplied to the engine. This is useful to get the values of the search space in a simple dictionary format, or as a list of values using `parameters.values()`.

In [4]:
# define the evaluation function
def squared_error_loss(id, parameters):
    x = parameters['x']
    y = parameters['y']
    y_sample = 2 * x - y

    return np.square(y_sample - value)

# Budgeted computation

PySHAC is built for limiting the amount of computation required to arrive at a useful result. We declare these budgets with two variables - 

1) `total budget`: Maximum number of evaluations that the pyshac engine will perform. This is the total number of evaluations, and is useful to set this budget when each evaluation is expensive. Consider having to train a new model for 20 epochs each time to obtain its accuracy as the evaluation metric.

2) `num batches`: Declares how to divide the entire search space into batches. In doing so, you also define how many times the search space will be halved.

3) Normally, the objective function passed is considered to be a loss function, which is minimized. We can also pass a directive to the engine, to maximize the objective function instead.

4) Maximum number of classifiers that should be used. In certain cases, the search space is not large enough to warrant the upper limit of 18 classifiers (which divides the search space 2 ^ 18 times).

In [7]:
# define the total budget as 100 evaluations
total_budget = 100  # 100 evaluations at maximum

# define the number of batches
num_batches = 10  # 10 samples per batch

# define the objective
objective = 'min'  # minimize the squared loss

max_classifiers = 18

# PySHAC Engine
There are several engines available to PySHAC, which should be used in specific cases or task that is being performed. PySHAC has support for Tensorflow, Keras and Numpy / PyTorch libraries.

An engines task is to sequentially halve the search space after each batch such that the objective function is optimized. For that it requires the above 4 parameters. 

In [8]:
shac = pyshac.SHAC(parameters, total_budget, num_batches, objective, max_classifiers)

Number of workers possible : 10
Using 10 parallel workers, it will require 10 epochs to fit 9 classifiers.
Each classifier will be provided 10 samples to train per epoch.


d:\users\yue\pycharmprojects\shac\pyshac\core\engine.py:467: UserWarning: Number of workers exceeds 8 cores on device. Reducing parallel number of cores used to prevent resource starvation.
  "number of cores used to prevent resource starvation." % (cpu_count))


# Training the PySHAC Engine

PySHAC is inspired by the Scikit-learn framework, and has simple functions to train and evaluate the engine.

The `fit` method has 3 options which allow faster computation in exchange for some loss in the evaluation performance.

1) `skip_cv_checks`: Usually, PySHAC will perform 5 fold cross validation on the results of each batch to get a more robust model. However, this may sometimes fail if the results of a batch are not discriminative enough to allow 5 fold training. In such a case, that entire batch will be not be used to train a classifier, and the engine moves onto the next batch. To prevent this wastage, this flag can be set to False. However, note that this will reduce the performance of the engine significantly in finding good hyper parameter combinations from the search space.

2) `early_stop`: If at any point the engine fails to train a new classifier, there is a high chance that it may have reduced the search space as much as it could and may fail at the subsequent rounds as well. You can optionally stop training to prevent wasted computations.

3) `relax_checks`: If the number of classifiers is large enough, the engine may fail to reduce the search space any longer, in which case it can either stop early or find more batch samples to try again. Another option is to simply relax the check that each subsequent classifier reduce the space further, and simply accept the classifier.

In [9]:
shac.fit(squared_error_loss, skip_cv_checks=True, early_stop=False, relax_checks=False)

Training with 8 generator (loky backend) and 8 evaluator threads (loky backend) with a batch size of 10
Beginning epoch 0001 out of 0010
Number of classifiers availale = 0 (1 samples generated per accepted sample on average)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    9.8s remaining:   23.0s
[Parallel(n_jobs=8)]: Done   5 out of  10 | elapsed:    9.8s remaining:    9.8s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    9.8s remaining:    4.1s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    9.8s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0380s.) Setting batch_size=10.
[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.0s remaining: 

Finished generating 10 samples
Finished evaluating 10 samples
Finished training the 1-th classifier
Serializing dataset...
Serialization of dataset done !


Finished training    1 out of   10 epochs

Serializing data and models
Serializing dataset...
Serialization of dataset done !
Saved classifier #1

Beginning epoch 0002 out of 0010
Number of classifiers availale = 1 (2 samples generated per accepted sample on average)
Finished generating 10 samples
Finished evaluating 10 samples
Finished training the 2-th classifier
Serializing dataset...
Serialization of dataset done !


Finished training    2 out of   10 epochs

Serializing data and models
Serializing dataset...
Serialization of dataset done !
Saved classifier #2

Beginning epoch 0003 out of 0010
Number of classifiers availale = 2 (4 samples generated per accepted sample on average)
Finished generating 10 samples


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0080s.) Setting batch_size=48.
[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0050s.) Setting batch_size=78.
[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Paralle

Finished evaluating 10 samples
Finished training the 3-th classifier
Serializing dataset...
Serialization of dataset done !


Finished training    3 out of   10 epochs

Serializing data and models
Serializing dataset...
Serialization of dataset done !
Saved classifier #3

Beginning epoch 0004 out of 0010
Number of classifiers availale = 3 (8 samples generated per accepted sample on average)
Finished generating 10 samples
Finished evaluating 10 samples
Finished training the 4-th classifier
Serializing dataset...
Serialization of dataset done !


Finished training    4 out of   10 epochs

Serializing data and models
Serializing dataset...
Serialization of dataset done !
Saved classifier #4

Beginning epoch 0005 out of 0010
Number of classifiers availale = 4 (16 samples generated per accepted sample on average)


...

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0400s.) Setting batch_size=8.
[Parallel(n_jobs=16)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=16)]: Done   5 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=16)]: Done   7 out of  10 | elapsed:    1.4s remaining:    0.5s
[Parallel(n_jobs=16)]: Done  10 out of  10 | elapsed:    3.3s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0030s.) Setting batch_size=1066.
[Parallel(n_jobs=16)]: Done  10 out of  10 | elapsed:    0.0s finished


Finished generating 10 samples


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Finished evaluating 10 samples
Finished training the 10-th classifier
Cannot train any more models as maximum number of models has been reached.
Serializing dataset...
Serialization of dataset done !


Finished training   10 out of   10 epochs

Serializing data and models
Serializing dataset...
Serialization of dataset done !
Saved classifier #9

Finished training all models !


[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    4.3s remaining:   10.2s
[Parallel(n_jobs=8)]: Done   5 out of  10 | elapsed:    4.3s remaining:    4.3s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    4.4s remaining:    1.8s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    4.4s finished


# Restoring the engine

Once PySHAC has trained an engine, it serializes the information and the classifiers to a directory called `shac` at the root of the directory where it was run.

You could then construct an empty engine, and restore the values to that engine from that directory.

In [10]:
shac.restore_data()


Found and restored dataset containing 100 samples
Found and restored 9 classifiers



# Predicting good hyper parameters

After training, the engine can be used to sample good hyper parameters from the search space. However there is something to note: 

As the number of classifiers increases, the average number of samples from the search space that is required before the engine finds a good parameter is close to 2 ^ (number of classifiers). 

If you use all 18 classifiers, it can take a while even for a single sample to be obtained !

Therefore, if you don't need all of the classifiers for the best predictions, you can use `max_classifiers` parameter in `predict` to obtain a good enough fit.

In [11]:
# sample more than one batch of hyper parameters
parameter_samples = shac.predict(20)  # samples 20 hyper parameters

Evaluating 3 batches (for a total of 20 samples) with 8 generator (loky backend)
Number of classifiers availale = 9 (512 samples generated per accepted sample on average)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   10.2s remaining:   30.7s
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   10.2s remaining:   17.1s
[Parallel(n_jobs=8)]: Done   4 out of   8 | elapsed:   10.3s remaining:   10.3s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   10.3s remaining:    6.1s
[Parallel(n_jobs=8)]: Done   6 out of   8 | elapsed:   10.3s remaining:    3.4s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   10.3s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   10.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   4 out of   8 | elapsed:    0.0s

# Lets evaluate the predicted samples

In [13]:
losses = [squared_error_loss(0, params) for params in parameter_samples]
x_list = [param['x'] for param in parameter_samples]
y_list = [param['y'] for param in parameter_samples]

for i, (x, y) in enumerate(zip(x_list, y_list)):
    print("Sample %d : (%0.4f, %0.4f). f(x, y) = %0.4f" % (i + 1, x, y, 2. * x - y))

print()
print("Mean squared error of samples : ", np.mean(losses))


Sample 1 : (1.7043, -0.2452). f(x, y) = 3.6538
Sample 2 : (1.3181, -1.1737). f(x, y) = 3.8098
Sample 3 : (1.4302, -0.9667). f(x, y) = 3.8270
Sample 4 : (1.6817, -1.0739). f(x, y) = 4.4373
Sample 5 : (1.4596, -0.6395). f(x, y) = 3.5586
Sample 6 : (1.2671, -0.3822). f(x, y) = 2.9165
Sample 7 : (1.6809, -1.0600). f(x, y) = 4.4218
Sample 8 : (1.3511, -0.4537). f(x, y) = 3.1560
Sample 9 : (1.3414, -0.8756). f(x, y) = 3.5583
Sample 10 : (1.4383, -0.8550). f(x, y) = 3.7316
Sample 11 : (1.3617, -0.5072). f(x, y) = 3.2306
Sample 12 : (1.7481, -0.5372). f(x, y) = 4.0334
Sample 13 : (1.7414, -0.7551). f(x, y) = 4.2379
Sample 14 : (1.3671, -1.2492). f(x, y) = 3.9833
Sample 15 : (1.6071, -0.7078). f(x, y) = 3.9220
Sample 16 : (1.4999, -0.5197). f(x, y) = 3.5195
Sample 17 : (1.4771, -0.4925). f(x, y) = 3.4467
Sample 18 : (1.4425, -1.3390). f(x, y) = 4.2240
Sample 19 : (1.6438, -1.3274). f(x, y) = 4.6150
Sample 20 : (1.7102, -0.0834). f(x, y) = 3.5037

Mean squared error of samples :  0.2385559014882